In [ ]:
!git clone https://github.com/pangdatangtt/UniNet.git

Cloning into 'UniNet'...
remote: Enumerating objects: 386, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 386 (delta 112), reused 66 (delta 66), pack-reused 255 (from 2)
Receiving objects: 100% (386/386), 6.63 MiB | 12.79 MiB/s, done.
Resolving deltas: 100% (186/186), done.


In [8]:
# Önce varsa eski linki sil (opsiyonel)
!rm -rf /content/UniNet/data

# Yeni bağlantıyı oluştur
!ln -s "/content/drive/MyDrive/Wood_Dataset" "/content/UniNet/data"




In [27]:
!python '/content/UniNet/main.py' \
  --dataset "MVTec AD" \
  --setting oc \
  --train_and_test_all \
  --is_saved \
  --save_dir "./results" \
  --epoch 100


training on MVTec AD dataset (separate-class)
class:wood, domain:industrial, setting:oc, dataset:MVTec AD, epochs:100, batch_size:8, image_size:256, center_crop:256, lr_s:0.005, lr_t:1e-06, T:2, weighted_decision_mechanism:True, default:0.3, alpha:0.01, beta:3e-05, train_and_test_all:True, is_saved:True, save_dir:./results, load_ckpts:False, 
cuda
epoch [1/100], loss:20.4108
epoch [2/100], loss:19.7393
epoch [3/100], loss:19.5118
epoch [4/100], loss:19.3994
epoch [5/100], loss:19.3333
epoch [6/100], loss:19.2677
epoch [7/100], loss:19.2251
epoch [8/100], loss:19.1838
epoch [9/100], loss:19.1584
epoch [10/100], loss:19.1295
🕒 FPS: 12.84
✅ gt_mask max: True, anomaly_map max: 3.7661192417144775
🔍 Best threshold (F1-max): 2.7200
📊 Confusion Matrix (Image-Level):
 [[ 1 69]
 [ 1 70]]
✅ F1 Score: 0.667 | Accuracy: 0.504
🧩 pred_masks unique values: [0 1]
📏 Mean IoU (Defect Images Only): 0.0450
Sample Auroc: 62.0, Pixel Auroc: 79.4, Pixel Aupro: 14.9
modules have been saved to ./ckpts/MVTec AD/

In [28]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

def debug_predict(image_tensor, model, device):
    image_tensor = image_tensor.to(device)

    with torch.no_grad():
        output = model(image_tensor)

        # 🔍 Model çıktısının yapısını incele
        if isinstance(output, tuple) and isinstance(output[0], list):
            anomaly_map = output[0][0]
        else:
            raise ValueError("⚠ Model çıktısı tuple[list[tensor]] yapısında değil.")

        if not isinstance(anomaly_map, torch.Tensor):
            raise TypeError(f"❌ anomaly_map bir Tensor değil: {type(anomaly_map)}")

        # 🔍 Anomaly map işleme
        anomaly_map = anomaly_map.mean(dim=1, keepdim=True)
        anomaly_map = torch.nn.functional.interpolate(anomaly_map, size=(256, 256), mode="bilinear", align_corners=False)
        anomaly_map = anomaly_map.clamp(0, 1).cpu().squeeze().numpy()

        # 🔍 Skor analizleri
        raw_max = float(anomaly_map.max())
        raw_mean = float(anomaly_map.mean())
        raw_percentile = float(np.percentile(anomaly_map, 99))

        # 🔍 DFS ağırlıkları varsa kontrol et
        if hasattr(model, 'dfs'):
            try:
                print("theta1 mean:", model.dfs.theta1.mean().item())
                print("theta2 mean:", model.dfs.theta2.mean().item())
                print("theta3 mean:", model.dfs.theta3.mean().item())
            except Exception as e:
                print("(⚠ DFS parametreleri okunamadı)", e)

        print(f"\n🧠 Skorlar:\n - Max: {raw_max:.4f}\n - Mean: {raw_mean:.4f}\n - 99th %: {raw_percentile:.4f}")

        # 🔍 Karar
        threshold = 0.15  # yeni düşük eşik
        label = "Anomali" if raw_max > threshold else "Normal"
        print(f"🔎 Tahmin: {label} (eşik: {threshold})")

        # 🔍 Haritayı görselleştir
        plt.imshow(anomaly_map, cmap='jet')
        plt.colorbar()
        plt.title(f"Anomaly Map\nMax: {raw_max:.3f}, Mean: {raw_mean:.3f}, 99%: {raw_percentile:.3f}\nLabel: {label}")
        plt.tight_layout()
        plt.show()

        return anomaly_map, raw_max, label




import torch

# BEST_P_PRO.pth dosyasının yolunu belirt
ckpt_path = "/content/ckpts/MVTec AD/wood/BEST_P_PRO.pth"  # Gerekirse tam yolu düzelt

checkpoint = torch.load(ckpt_path, map_location="cpu")

print("📦 Checkpoint tipi:", type(checkpoint))

# Eğer dict ise:
if isinstance(checkpoint, dict):
    print("🔑 Anahtarlar:", list(checkpoint.keys())[:10])

    # Eğer doğrudan dfs.weight gibi şeyler varsa
    if all(isinstance(v, torch.Tensor) for v in checkpoint.values()):
        print("✅ Doğrudan state_dict (tek düzeyli)")
        dfs_weights = {k: v.shape for k, v in checkpoint.items() if 'dfs' in k.lower()}
        print("🔍 DFS Ağırlıkları:", dfs_weights)

    # Eğer alt modüller varsa (dict içinde dict)
    elif all(isinstance(v, dict) for v in checkpoint.values()):
        print("✅ dict içinde dict (modül bazlı kaydedilmiş)")
        for module_name in checkpoint.keys():
            print(f"📂 {module_name}: {list(checkpoint[module_name].keys())[:5]}")

else:
    print("❌ Beklenmeyen format:", type(checkpoint))


📦 Checkpoint tipi: <class 'dict'>
🔑 Anahtarlar: ['tt', 'bn', 'st', 'dfs']
✅ dict içinde dict (modül bazlı kaydedilmiş)
📂 tt: ['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var']
📂 bn: ['bn_layer.0.0.bn2.weight', 'bn_layer.0.0.bn2.bias', 'bn_layer.0.0.bn2.running_mean', 'bn_layer.0.0.bn2.running_var', 'bn_layer.0.0.bn2.num_batches_tracked']
📂 st: ['layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var']
📂 dfs: ['theta1', 'theta2', 'theta3']


In [29]:
!python '/content/UniNet/main.py' --dataset "MVTec AD" --setting oc --save_dir "./results" --load_ckpts

Exception ignored in: <function _get_module_lock.<locals>.cb at 0x78d192913240>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 207, in cb
KeyboardInterrupt: 
Traceback (most recent call last):
  File "/content/UniNet/main.py", line 4, in <module>
    from train_unsupervisedAD import train
  File "/content/UniNet/train_unsupervisedAD.py", line 10, in <module>
    from datasets import loading_dataset
  File "/content/UniNet/datasets.py", line 1, in <module>
    from torchvision import transforms
  File "/usr/local/lib/python3.11/dist-packages/torchvision/__init__.py", line 10, in <module>
    from torchvision import _meta_registrations, datasets, io, models, ops, transforms, utils  # usort:skip
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torchvision/models/__init__.py", line 2, in <module>
    from .convnext import *
  File "/usr/local/lib/python3.11/dist-pac

In [30]:
def test(c, suffix='BEST_P_PRO'):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"🖥️  Device: {device}")

    dataset_name = c.dataset

    # 🔁 SENİN DOĞRUDAN VERDİĞİN CHECKPOINT PATH
    ckpt_path = "/content/UniNet/ckpts/ckpts/MVTec AD/wood"  # ← burada BEST_P_PRO.pth var

    # Dataset yükle
    dataset_info = loading_dataset(c, dataset_name)
    test_dataloader = dataset_info[1]

    # Model tanımı ve ağırlık yükleme
    Source_teacher, bn = wide_resnet50_2(c, pretrained=True)
    Source_teacher.layer4 = None
    Source_teacher.fc = None
    student = de_wide_resnet50_2(pretrained=False)
    DFS = DomainRelated_Feature_Selection()
    [Source_teacher, bn, student, DFS] = to_device([Source_teacher, bn, student, DFS], device)
    Target_teacher = copy.deepcopy(Source_teacher)

    new_state = load_weights([Target_teacher, bn, student, DFS], ckpt_path, suffix)
    Target_teacher = new_state['tt']
    bn = new_state['bn']
    student = new_state['st']
    DFS = new_state['dfs']

    model = UniNet(c, Source_teacher.cuda().eval(), Target_teacher, bn, student, DFS)

    # Performans metriği hesapla
    auroc_px, auroc_sp, pro = evaluation_indusAD(c, model, test_dataloader, device)

    return auroc_sp, auroc_px, pro, model, device


In [31]:
def test_all_images(c, model, device):
    import glob
    import os
    from PIL import Image
    import numpy as np
    import matplotlib.pyplot as plt
    import cv2
    import datetime
    from torchvision import transforms as T
    from scipy.ndimage import gaussian_filter
    import torch
    from torch.nn import functional as F

    test_images_dir = f"/content/UniNet/data/mvtec/{c._class_}/test/defect/"
    gt_dir = f"/content/UniNet/data/mvtec/{c._class_}/ground_truth/defect/"

    image_paths = sorted(glob.glob(os.path.join(test_images_dir, "*.jpg")))
    os.makedirs("all_results", exist_ok=True)

    for img_path in image_paths:
        filename = os.path.basename(img_path)
        gt_path = os.path.join(gt_dir, filename.replace(".jpg", "_mask.jpg"))

        original_image = Image.open(img_path).convert('RGB')
        transform = T.Compose([
            T.Resize((c.image_size, c.image_size)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        input_tensor = transform(original_image).unsqueeze(0).to(device)

        model.train_or_eval(type='eval')
        n = model.n
        output_list = [list() for _ in range(n * 3)]

        with torch.no_grad():
            t_tf, de_features = model(input_tensor)
            for l, (t, s) in enumerate(zip(t_tf, de_features)):
                output = 1 - F.cosine_similarity(t, s)
                output_list[l].append(output)

            anomaly_score, anomaly_map = weighted_decision_mechanism(1, output_list, c.alpha, c.beta)
            anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            anomaly_map = anomaly_map.squeeze()

        # Normalize heatmap
        anomaly_map_norm = cv2.normalize(anomaly_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        heatmap = cv2.applyColorMap(anomaly_map_norm, cv2.COLORMAP_JET)

        # Contours
        _, thresh = cv2.threshold(anomaly_map_norm, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        original_cv = np.array(original_image.resize((c.image_size, c.image_size)))
        original_cv = cv2.cvtColor(original_cv, cv2.COLOR_RGB2BGR)
        contour_image = cv2.drawContours(original_cv.copy(), contours, -1, (0, 255, 0), 2)

        # GT mask
        if os.path.exists(gt_path):
            gt_mask = Image.open(gt_path).convert('L').resize((c.image_size, c.image_size))
        else:
            gt_mask = np.zeros_like(anomaly_map_norm)

        # Görseli Kaydet
        label = "defect" if "defect" in img_path.lower() else "good"
        img_number = filename.replace(".jpg", "")
        save_path = f"all_result/result_{label}_{img_number}.png"

        plt.figure(figsize=(15, 5))
        plt.subplot(1, 4, 1); plt.title("Original"); plt.imshow(original_image.resize((c.image_size, c.image_size))); plt.axis('off')
        plt.subplot(1, 4, 2); plt.title("Anomaly Map"); plt.imshow(anomaly_map, cmap='jet'); plt.axis('off')
        plt.subplot(1, 4, 3); plt.title("Contours"); plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB)); plt.axis('off')
        plt.subplot(1, 4, 4); plt.title("Ground Truth"); plt.imshow(gt_mask, cmap='gray'); plt.axis('off')
        plt.savefig(save_path)
        plt.close()

        print(f"[✓] {filename} -> Max Score: {anomaly_map.max().item():.4f}, saved to {save_path}")




In [ ]:
def test_all_images_gd(c, model, device):
    import glob
    import os
    from PIL import Image
    import numpy as np
    import matplotlib.pyplot as plt
    import cv2
    import datetime
    from torchvision import transforms as T
    from scipy.ndimage import gaussian_filter
    import torch
    from torch.nn import functional as F

    test_images_dir = f"/content/UniNet/data/mvtec/{c._class_}/test/*/"
    gt_dir = f"/content/UniNet/data/mvtec/{c._class_}/ground_truth/"

    image_paths = sorted(glob.glob(os.path.join(test_images_dir, "*.jpg")))
    os.makedirs("all_results", exist_ok=True)

    print(f"Toplam {len(image_paths)} test görüntüsü bulundu.")

    # 1️⃣ Tüm anomaly map max skorlarını topla
    max_scores = []
    anomaly_maps = []
    processed = []

    for img_path in image_paths:
        filename = os.path.basename(img_path)

        original_image = Image.open(img_path).convert('RGB')
        transform = T.Compose([
            T.Resize((c.image_size, c.image_size)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        input_tensor = transform(original_image).unsqueeze(0).to(device)

        model.train_or_eval(type='eval')
        n = model.n
        output_list = [list() for _ in range(n * 3)]

        with torch.no_grad():
            t_tf, de_features = model(input_tensor)
            for l, (t, s) in enumerate(zip(t_tf, de_features)):
                output = 1 - F.cosine_similarity(t, s)
                output_list[l].append(output)

            anomaly_score, anomaly_map = weighted_decision_mechanism(1, output_list, c.alpha, c.beta)
            anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            anomaly_map = anomaly_map.squeeze()

        max_score = anomaly_map.max().item()
        label = "defect" if max_score >= 1.98 else "good"

        max_scores.append(max_score)
        anomaly_maps.append(anomaly_map)
        processed.append((img_path, original_image, anomaly_map, label, max_score))

    global_max = max(max_scores)
    print(f"\n🌡️ Global max anomaly score: {global_max:.4f}\n")

    # 2️⃣ Normalize edip görselleri kaydet
    for idx, (img_path, original_image, anomaly_map, label, max_score) in enumerate(processed):
        filename = os.path.basename(img_path)
        gt_path = os.path.join(gt_dir, label, filename.replace(".jpg", "_mask.jpg"))

        anomaly_map_norm = (anomaly_map / global_max * 255).astype(np.uint8)
        heatmap = cv2.applyColorMap(anomaly_map_norm, cv2.COLORMAP_JET)

        _, thresh = cv2.threshold(anomaly_map_norm, 127, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        original_cv = np.array(original_image.resize((c.image_size, c.image_size)))
        original_cv = cv2.cvtColor(original_cv, cv2.COLOR_RGB2BGR)
        contour_image = cv2.drawContours(original_cv.copy(), contours, -1, (0, 255, 0), 2)

        # GT mask
        if os.path.exists(gt_path):
            gt_mask = Image.open(gt_path).convert('L').resize((c.image_size, c.image_size))
        else:
            gt_mask = np.zeros_like(anomaly_map_norm)

        save_path = f"all_results/result_{label}_{filename.replace('.jpg', '')}.png"
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 4, 1); plt.title("Original"); plt.imshow(original_image.resize((c.image_size, c.image_size))); plt.axis('off')
        plt.subplot(1, 4, 2); plt.title("Anomaly Map"); plt.imshow(anomaly_map, cmap='jet'); plt.axis('off')
        plt.subplot(1, 4, 3); plt.title("Contours"); plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB)); plt.axis('off')
        plt.subplot(1, 4, 4); plt.title("Ground Truth"); plt.imshow(gt_mask, cmap='gray'); plt.axis('off')
        plt.savefig(save_path)
        plt.close()

        print(f"[{idx+1}/{len(processed)}] ✓ {filename} → Max Score: {max_score:.4f}, Predicted: {label}, Saved to: {save_path}")

auroc_sp, auroc_px, pro, model, device = test(c)
test_all_images_gd(c, model, device)


In [ ]:
!rm -rf /content/UniNet/all_results